In [ ]:
from models_impute import *
from pygrinder import block_missing

In [ ]:
data = pd.read_csv("./data/lorenz/lorenz_dataset_0_timeseries.csv",header=None)
data = data.values

In [ ]:
cg = pd.read_csv('./causality_matrices/lorenz_causality_matrix.csv', header=None)
cg = cg.values
model_params = {
        'num_levels': 10,
        'kernel_size': 8,
        'dilation_c': 2
    }

In [ ]:
data = block_missing(data[np.newaxis,...],factor=0.1, block_len=3, block_width=3)
data = data[0]
data

In [ ]:
data_imputed = impute(data, cg, model_params)
data_imputed

In [ ]:
# test_miracle.py
import numpy as np
from baseline import miracle_impu

# 创建测试数据
test_data = np.random.randn(100, 10)
test_data[np.random.random((100, 10)) < 0.1] = np.nan

print("测试数据形状:", test_data.shape)
print("缺失值数量:", np.isnan(test_data).sum())

try:
    result = miracle_impu(test_data)
    print("MIRACLE结果形状:", result.shape if result is not None else "None")
    print("MIRACLE结果类型:", type(result))
    
    if result is not None:
        print("填补后缺失值:", np.isnan(result).sum())
    else:
        print("❌ MIRACLE返回了None")
        
except Exception as e:
    print(f"❌ MIRACLE测试失败: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
import pandas as pd
import numpy as np

# 读取 CSV 文件
zero_matrix = pd.read_csv("zero_impu_matrix.csv").values  # shape: (T, N)

# 构造掩码矩阵：0 的位置为 0，其余为 1
mask_matrix = np.where(zero_matrix == 0, 0, 1)

# 保存新矩阵到 CSV
pd.DataFrame(mask_matrix).to_csv("zero_impu_mask.csv", index=False)


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pred = pd.read_csv('./train_result.csv').values
gt = pd.read_csv('./gt_matrix.csv').values

In [ ]:
import torch.nn.functional as F
import torch

In [ ]:
mask_matrix = 1 - mask_matrix

In [ ]:
pred*mask_matrix

In [ ]:
gt*mask_matrix

In [ ]:
res = F.mse_loss(torch.tensor(pred*mask_matrix), torch.tensor(gt*mask_matrix))
res

In [ ]:
import tempfile
import os

tmp_file = tempfile.NamedTemporaryFile(delete=False)
tmp_file.close()
os.remove(tmp_file.name)


In [ ]:
import os
import shutil

# 原始目录和目标目录路径
src_dir = './data/III'        # 替换为你的源目录路径
dst_dir = './data/mimic-iii'   # 替换为你的目标目录路径

# 创建目标目录（如果不存在）
os.makedirs(dst_dir, exist_ok=True)

# 获取源目录中的所有文件名（按名称排序，可改为按修改时间等）
all_files = sorted(f for f in os.listdir(src_dir) if os.path.isfile(os.path.join(src_dir, f)))

# 选择前100个文件
files_to_move = all_files[:100]

# 移动文件
for fname in files_to_move:
    shutil.move(os.path.join(src_dir, fname), os.path.join(dst_dir, fname))

print(f"已成功移动 {len(files_to_move)} 个文件到 {dst_dir}")


In [ ]:
import os
import pandas as pd
import numpy as np
from baseline import *
from models_impute import *
# 指定目录
data_dir = "./data/III"  # 替换为你的目录路径

# 获取目录下第一个 CSV 文件
csv_files = [f for f in os.listdir(data_dir) if f.endswith(".csv")]
assert csv_files, "目录中没有找到 CSV 文件"
csv_path = os.path.join(data_dir, csv_files[0])

# 读取数据
df = pd.read_csv(csv_path)
mx = df.values.astype(np.float32)

# 记录原始缺失位置
nan_pos = np.argwhere(np.isnan(mx))

# 填补缺失值
imputed_mx = timesnet_impu(mx,)

# 打印填补前后的值（仅缺失位置）
print(f"填补前后对比（仅缺失位置）：{imputed_mx}")



In [ ]:
import pkgutil
import sys

for _, name, _ in pkgutil.iter_modules():
    if 'tsde' in name:
        print(name)


In [ ]:
import os
import numpy as np
import pandas as pd
from pygrinder import mcar, mar_logistic, mnar_x

def dig_missing_values(
    file_path: str,
    output_dir: str,
    obs_rate: float = 0.6,
    mar_missing_rate: float = 0.4,
    mnar_offset: float = 0.7,
    mcar_p: float = 0.1
):
    # 读取数据
    df = pd.read_csv(file_path)
    data = df.values.astype(np.float32)
    
    # 应用缺失机制
    X = mar_logistic(data, obs_rate=obs_rate, missing_rate=mar_missing_rate)
    X = X[np.newaxis, ...]                  # 添加 batch 维度
    X = mnar_x(X, offset=mnar_offset)       # MNAR 缺失
    X = mcar(X, p=mcar_p)                   # 再次添加 MCAR 缺失
    X = X.squeeze(0)                        # 去除 batch 维度

    # 保存结果
    filename = os.path.basename(file_path).replace('.csv', '_dig_missing.csv')
    output_path = os.path.join(output_dir, filename)
    pd.DataFrame(X).to_csv(output_path, index=False)
    print(f"✅ 保存至: {output_path}")
dig_missing_values(
    file_path='./data/air/2013-03-07.csv',
    output_dir='./',)

In [ ]:
import tsdb

# list all available datasets in TSDB
tsdb.list()
# ['physionet_2012',
#  'physionet_2019',
#  'electricity_load_diagrams',
#  'beijing_multisite_air_quality',
#  'italy_air_quality',
#  'vessel_ais',
#  'electricity_transformer_temperature',
#  'pems_traffic',
#  'solar_alabama',
#  'ucr_uea_ACSF1',
#  'ucr_uea_Adiac',
#  ...

tsdb.download_and_extract('beijing_multisite_air_quality', './save_it_here')


In [ ]:
import os
import pandas as pd

# ==== 用户自定义路径 ====
input_file = './save_it_here/PRSA_Data_20130301-20170228/PRSA_Data_Aotizhongxin_20130301-20170228.csv'       # 输入文件路径
output_dir = './data/air'     # 输出文件夹路径

# ==== 需要提取的列 ====
columns_needed = ["year", "month", "day", "hour", "PM2.5", "PM10", "SO2", "NO2", "CO",
                  "O3", "TEMP", "PRES", "DEWP", "RAIN", "WSPM"]

# ==== 创建输出目录（如果不存在） ====
os.makedirs(output_dir, exist_ok=True)

# ==== 读取数据并提取指定列 ====
df = pd.read_csv(input_file, usecols=columns_needed)

# ==== 按日期分组（year, month, day） ====
grouped = df.groupby(['year', 'month', 'day'])

# ==== 遍历每一天的数据 ====
for (y, m, d), group in grouped:
    # 检查是否为整天（24小时）且无缺失值
    if len(group) == 24 and not group.isnull().values.any():
        # 构造保存文件名
        filename = f"{y:04d}-{m:02d}-{d:02d}.csv"
        save_path = os.path.join(output_dir, filename)
        group.to_csv(save_path, index=False,)
        print(f"✅ 已保存: {filename}")

print("🎉 所有完整天数据提取完成！")


In [ ]:
import os

def delete_last_n_files(folder_path, n, sort_by='name'):
    """
    删除指定文件夹下的最后 n 个文件。
    
    :param folder_path: 文件夹路径
    :param n: 要删除的文件个数
    :param sort_by: 'name' 或 'mtime'，按文件名或最后修改时间排序
    """
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)
             if os.path.isfile(os.path.join(folder_path, f))]
    
    if sort_by == 'name':
        files.sort()  # 按文件名排序
    elif sort_by == 'mtime':
        files.sort(key=os.path.getmtime)  # 按修改时间排序
    else:
        raise ValueError("sort_by must be 'name' or 'mtime'")

    to_delete = files[-n:]  # 取最后 n 个文件

    for file_path in to_delete:
        try:
            os.remove(file_path)
            print(f"✅ 已删除: {file_path}")
        except Exception as e:
            print(f"❌ 删除失败: {file_path}, 错误: {e}")

# 示例用法
delete_last_n_files("./data/air", n=575, sort_by='name')  # 或 sort_by='mtime'


In [ ]:
import os
import shutil

def copy_matching_files(dir_a, dir_b, dir_c):
    if not os.path.exists(dir_c):
        os.makedirs(dir_c)

    # 获取目录 B 中所有文件名（不含路径）
    b_filenames = set(os.listdir(dir_b))

    # 遍历目录 A，查找与目录 B 同名的文件
    for filename in os.listdir(dir_a):
        if filename in b_filenames:
            src_path = os.path.join(dir_a, filename)
            dst_path = os.path.join(dir_c, filename)
            shutil.copy2(src_path, dst_path)
            print(f"✅ 已复制: {filename}")

# ✅ 示例使用：
dir_a = "./data/III"# 替换为实际路径
dir_b = "./data_imputed/my_model/III"
dir_c = "./data/downstreamIII"  # 替换为实际路径
copy_matching_files(dir_a, dir_b, dir_c)


In [ ]:
import os
import pandas as pd

def pad_csv_files_to_193_rows(directory):
    for filename in os.listdir(directory):
        if not filename.endswith(".csv"):
            continue  # 跳过非 CSV 文件

        filepath = os.path.join(directory, filename)

        try:
            df = pd.read_csv(filepath)

            original_rows = len(df)
            target_rows = 192  # 不包含表头的部分

            if original_rows < target_rows:
                # 获取最后一行（如果为空则跳过）
                if original_rows == 0:
                    print(f"⚠️ 文件 {filename} 是空的，跳过")
                    continue

                last_row = df.iloc[[-1]]  # 保持 DataFrame 格式
                rows_to_add = target_rows - original_rows

                # 重复追加最后一行
                df = pd.concat([df] + [last_row] * rows_to_add, ignore_index=True)
                df.to_csv(filepath, index=False)
                print(f"✅ 文件 {filename} 补齐至 {target_rows+1} 行（含表头）")

            elif original_rows > target_rows:
                print(f"✅ 文件 {filename} 已满足行数要求 ({original_rows+1} 行含表头)")

            else:
                print(f"✅ 文件 {filename} 恰好 {target_rows+1} 行（含表头）")

        except Exception as e:
            print(f"❌ 处理文件 {filename} 出错: {e}")

# ✅ 示例用法
target_dir = "./data_imputed/my_model/III"  # 替换为实际目录路径
pad_csv_files_to_193_rows(target_dir)


In [ ]:
# saits_parallel.py
import os
import pandas as pd
import numpy as np
import torch
from multiprocessing import Process, Queue, current_process
from tqdm import tqdm

def saits_impu(mx, epochs=50, d_model=32, n_layers=2, n_heads=4,
               d_k=8, d_v=8, d_ffn=16, dropout=0.4, device=None):
    from pypots.imputation import SAITS

    global_mean = np.nanmean(mx)
    all_nan_cols = np.all(np.isnan(mx), axis=0)
    if all_nan_cols.any():
        print(f"发现 {all_nan_cols.sum()} 列全为NaN，这些列将用填充")
        mx[:, all_nan_cols] = global_mean

    mx = mx.copy()
    n_steps, n_features = mx.shape
    data_3d = mx[np.newaxis, :, :]

    saits = SAITS(
        n_steps=n_steps,
        n_features=n_features,
        n_layers=n_layers,
        d_model=d_model,
        n_heads=n_heads,
        d_k=d_k,
        d_v=d_v,
        d_ffn=d_ffn,
        dropout=dropout,
        epochs=epochs,
        device=device,
    )

    train_set = {"X": data_3d}
    saits.fit(train_set)
    imputed = saits.impute({"X": data_3d})
    return imputed[0]

def gpu_worker(file_queue, input_dir, output_dir, gpu_id):
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    pid = current_process().pid

    while not file_queue.empty():
        try:
            fname = file_queue.get_nowait()
        except:
            break

        try:
            fpath = os.path.join(input_dir, fname)
            mx = pd.read_csv(fpath).values.astype(np.float32)
            filled = saits_impu(mx, device=device)

            out_path = os.path.join(output_dir, fname)
            pd.DataFrame(filled).to_csv(out_path, index=False)
            print(f"[PID {pid} GPU {gpu_id}] ✅ 处理完成: {fname}")
        except Exception as e:
            print(f"[PID {pid} GPU {gpu_id}] ❌ 处理失败: {fname}，错误：{e}")

def parallel_saits_impute(input_dir, output_dir):
    import multiprocessing as mp

    os.makedirs(output_dir, exist_ok=True)
    file_list = [f for f in os.listdir(input_dir) if f.endswith('.csv')]
    file_queue = mp.Queue()
    for fname in file_list:
        file_queue.put(fname)

    num_gpus = torch.cuda.device_count()
    processes = []

    print(f"🚀 启动 {num_gpus} 个 GPU 并行填补，共 {len(file_list)} 个文件")

    for gpu_id in range(num_gpus):
        p = mp.Process(target=gpu_worker, args=(file_queue, input_dir, output_dir, gpu_id))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()

    print("✅ 所有文件处理完成！")

if __name__ == "__main__":
    import multiprocessing as mp
    mp.set_start_method("spawn", force=True)

    input_dir = "./data/downstreamIII"
    output_dir = "./data_imputed/saits/III"
    parallel_saits_impute(input_dir, output_dir)


🚀 启动 4 个 GPU 并行填补，共 1367 个文件
✅ 所有文件处理完成！


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/data/anaconda3/envs/Former/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/data/anaconda3/envs/Former/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'gpu_worker' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/data/anaconda3/envs/Former/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/data/anaconda3/envs/Former/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'gpu_worker' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/data/anaconda3/envs/Former/lib/py

: 

In [ ]:
import pandas as pd
import numpy as np
tar = pd.read_csv('./data/mimic/200068.csv').values
print(tar)
from baseline import *
res = knn_impu(tar, k=5)
print(res)